In [1]:
from datasets import load_dataset

# Load the ConvAI2 dataset
dataset = load_dataset("conv_ai_2", split="train", trust_remote_code=True)

# Save the dialogues to a text file
with open("ai_corpus.txt", "w", encoding="utf-8") as f:
    for dialog in dataset:
        for utterance in dialog["dialog"]:
            f.write(utterance["text"] + "\n")


In [1]:
from transformers import AutoModelForCausalLM, AutoTokenizer, Trainer, TrainingArguments, TextDataset, DataCollatorForLanguageModeling

model_name = "microsoft/DialoGPT-small"
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForCausalLM.from_pretrained(model_name)


def load_dataset(file_path, tokenizer):
    return TextDataset(
        tokenizer=tokenizer,
        file_path=file_path,
        block_size=128
    )

def fine_tune(model, dataset):
    training_args = TrainingArguments(
        output_dir='./results',
        overwrite_output_dir=True,
        num_train_epochs=3,  # Increase the number of epochs for better results
        per_device_train_batch_size=4,  # Adjust batch size based on your GPU capacity
        save_steps=10_000,
        save_total_limit=2,
    )
    data_collator = DataCollatorForLanguageModeling(
        tokenizer=tokenizer,
        mlm=False,
    )
    trainer = Trainer(
        model=model,
        args=training_args,
        data_collator=data_collator,
        train_dataset=dataset,
    )
    trainer.train()

# Load the dataset
dataset = load_dataset("ai_corpus.txt", tokenizer)
fine_tune(model, dataset)

model.save_pretrained("./fine_tuned_model")
tokenizer.save_pretrained("./fine_tuned_model")


/Users/sahilsharma/anaconda3/lib/python3.11/site-packages/transformers/utils/generic.py:260: FutureWarning: `torch.utils._pytree._register_pytree_node` is deprecated. Please use `torch.utils._pytree.register_pytree_node` instead.
  torch.utils._pytree._register_pytree_node(
/Users/sahilsharma/anaconda3/lib/python3.11/site-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(
/Users/sahilsharma/anaconda3/lib/python3.11/site-packages/transformers/utils/generic.py:260: FutureWarning: `torch.utils._pytree._register_pytree_node` is deprecated. Please use `torch.utils._pytree.register_pytree_node` instead.
  torch.utils._pytree._register_pytree_node(
/Users/sahilsharma/anaconda3/lib/python3.11/site-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and 

Step,Training Loss
500,3.104500
1000,2.442800
1500,2.224200
2000,2.143700
2500,2.039200


('./fine_tuned_model/tokenizer_config.json',
 './fine_tuned_model/special_tokens_map.json',
 './fine_tuned_model/vocab.json',
 './fine_tuned_model/merges.txt',
 './fine_tuned_model/added_tokens.json',
 './fine_tuned_model/tokenizer.json')

In [7]:
import gradio as gr
from transformers import AutoModelForCausalLM, AutoTokenizer

model_name = "./fine_tuned_model"
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForCausalLM.from_pretrained(model_name)

if tokenizer.pad_token is None:
    tokenizer.pad_token = tokenizer.eos_token

# Initialize chat history
chat_history = []

def chatbot(input_text):
    global chat_history
    
    # Append user input to history
    chat_history.append(f"You: {input_text}")
    
    # Process the input text
    inputs = tokenizer.encode_plus(input_text + tokenizer.eos_token, return_tensors="pt", padding=True)
    input_ids = inputs["input_ids"]
    attention_mask = inputs["attention_mask"]
    
    # Generate response
    chat_history_ids = model.generate(
        input_ids,
        attention_mask=attention_mask,
        max_length=100,
        pad_token_id=tokenizer.eos_token_id,
    )
    
    response = tokenizer.decode(chat_history_ids[:, input_ids.shape[-1]:][0], skip_special_tokens=True)
    
    # Append model response to history
    chat_history.append(f"Bot: {response}")
    
    return response

def show_history():
    return "\n".join(chat_history)

# Define Gradio interface
with gr.Blocks() as iface:
    with gr.Row():
        with gr.Column():
            chat_input = gr.Textbox(label="Type your message")
            submit_btn = gr.Button("Submit")
            history_btn = gr.Button("Show History")
            chat_output = gr.Textbox(label="Chatbot Response")
            history_output = gr.Textbox(label="Chat History")

            submit_btn.click(fn=chatbot, inputs=chat_input, outputs=chat_output)
            history_btn.click(fn=show_history, outputs=history_output)

iface.launch()


/Users/sahilsharma/anaconda3/lib/python3.11/site-packages/transformers/modeling_utils.py:479: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  return torch.load(checkpoint_file

Running on local URL:  http://127.0.0.1:7860

To create a public link, set `share=True` in `launch()`.


A decoder-only architecture is being used, but right-padding was detected! For correct generation results, please set `padding_side='left'` when initializing the tokenizer.


In [10]:
from flask import Flask, render_template, request, jsonify
from transformers import AutoModelForCausalLM, AutoTokenizer

app = Flask(__name__)

model_name = "./fine_tuned_model"
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForCausalLM.from_pretrained(model_name)

if tokenizer.pad_token is None:
    tokenizer.pad_token = tokenizer.eos_token

chat_history = []

@app.route('/')
def home():
    return render_template('simple_chatbot.html')

@app.route('/chat', methods=['POST'])
def chat():
    global chat_history
    input_text = request.json['input_text']
    
    chat_history.append(f"You: {input_text}")
    
    inputs = tokenizer.encode_plus(input_text + tokenizer.eos_token, return_tensors="pt", padding=True)
    input_ids = inputs["input_ids"]
    attention_mask = inputs["attention_mask"]
    
    chat_history_ids = model.generate(
        input_ids,
        attention_mask=attention_mask,
        max_length=100,
        pad_token_id=tokenizer.eos_token_id,
    )
    
    response = tokenizer.decode(chat_history_ids[:, input_ids.shape[-1]:][0], skip_special_tokens=True)
    chat_history.append(f"Bot: {response}")
    
    return jsonify({'response': response})

@app.route('/history', methods=['GET'])
def history():
    return jsonify({'history': "\n".join(chat_history)})


if __name__ == '__main__':
    app.run(debug=True, port=5002)


Using pad_token, but it is not set yet.


 * Serving Flask app '__main__'
 * Debug mode: on


 * Running on http://127.0.0.1:5002
Press CTRL+C to quit
 * Restarting with watchdog (fsevents)
0.00s - Debugger warning: It seems that frozen modules are being used, which may
0.00s - make the debugger miss breakpoints. Please pass -Xfrozen_modules=off
0.00s - to python to disable frozen modules.
0.00s - Note: Debugging will proceed. Set PYDEVD_DISABLE_FILE_VALIDATION=1 to disable this validation.
Traceback (most recent call last):
  File "<frozen runpy>", line 198, in _run_module_as_main
  File "<frozen runpy>", line 88, in _run_code
  File "/Users/sahilsharma/anaconda3/lib/python3.11/site-packages/ipykernel_launcher.py", line 17, in <module>
    app.launch_new_instance()
  File "/Users/sahilsharma/anaconda3/lib/python3.11/site-packages/traitlets/config/application.py", line 1074, in launch_instance
    app.initialize(argv)
  File "/Users/sahilsharma/anaconda3/lib/python3.11/site-packages/traitlets/config/application.py", line 118, in inner
    return method(app, *args, **kwargs)
   

SystemExit: 1

In [5]:
!pip install Flask

DEPRECATION: textract 1.6.5 has a non-standard dependency specifier extract-msg<=0.29.*. pip 24.1 will enforce this behaviour change. A possible replacement is to upgrade to a newer version of textract or contact the author to suggest that they release a version with a conforming dependency specifiers. Discussion can be found at https://github.com/pypa/pip/issues/12063
